In [1]:
import re
import pandas as pd
from string import ascii_uppercase 

In [2]:
#функция для подсчета суммы букв
letters = {letter: str(index) for index, letter in enumerate(ascii_uppercase, start=1)} 
def alphabet_position(text): 
    numbers = [int(letters[character]) for character in text if character in letters] 
    return sum(numbers)

names = open('names.txt')
for l in names:
    lines = sorted(re.sub(r"\"", "", l).split(","))
names.close
data = pd.DataFrame({'names':lines, 'sum':0}) #можнои без таблицы - но в ней выглядит лучше
data['sum'] = data.names.apply(alphabet_position) * (data.index + 1) #применяем функцию и сразу же умножаем на позицию
answer1 = sum(data['sum']) #Ответ задачи с именами

In [3]:
ip = dict() #словарь для хранения пар ip-кол-во

with open('hits.txt', 'r') as f:
    for row in f:
        ip_split = row.split('\t')[1]
        if not ip_split in ip: #если значения нет в словаре - плюсуем. иначе добавляем
            ip[ip_split] = 1 
        else:
            ip[ip_split] += 1
answer2 = max(ip, key=ip.get) #ответ задачи с ip адресами

In [4]:
#загрузим данные из таблиц, очистив их от NaN значений (в каждой таблице в начале и конце есть разделитель, которые дает NaN строку)

lesson = pd.read_csv('lessons.txt', sep= '|').dropna()
quality = pd.read_csv('quality.txt', sep= '|').dropna()
users = pd.read_csv('users.txt', sep= '|').dropna()
participants = pd.read_csv('participants.txt', sep= '|').dropna()

#очистим имена столбцов от лишних пробелов
lesson.columns = lesson.columns.str.replace(' ', '')
lesson.scheduled_time = pd.to_datetime(lesson.scheduled_time, errors = 'coerce').dt.to_period('D')
quality.columns = quality.columns.str.replace(' ', '')
users.columns = users.columns.str.replace(' ', '')
participants.columns = participants.columns.str.replace(' ', '')
#skipinitialspace

In [5]:
#Так как способа предзагрузки данных с разделителем и удалением лишних пробелов нет (документация, форумы), воспользуемся
#циклом, для очистки данных от лишних пробелов (в нашем случае - все пробелы, кроме scheduled_time лишние, в данных нет пробельной информации)
data_names = [lesson, quality, users, participants]
for i in data_names:
    columns_names = [j for j in i.columns]
    for j in columns_names:
        if i[j].dtypes == object:
            i[j] = i[j].apply(lambda x: re.sub(" ", "", x))
#Данное преобразование нужно для корректного и простого объединения таблиц
lesson.event_id = lesson.event_id.astype('int')
lesson = lesson.rename(columns = {'id':'lesson_id'})
participants.event_id = participants.event_id.astype('int')
users = users.rename(columns = {'id':'user_id'})

In [6]:
#Последний этап подготовки - сводная таблица.
group_data = lesson.merge(participants, on='event_id', copy = False)
group_data.shape
group_data = group_data.merge(quality, on='lesson_id', copy = False)
group_data.shape
#на данном шаге по непонятной причине дублируется в огромном количестве все данные, которые мы удалим следующим шагом
#возможно, воспринимает разные форматы? Пробелы? Но это не объясняет, почему в этом случае отрабатывает drop
group_data = group_data.merge(users, on='user_id')
group_data.shape
group_data = group_data.drop_duplicates().reset_index().drop(['index'], axis = 1)
group_data.shape
group_data.tech_quality = pd.to_numeric(group_data.tech_quality, errors='coerce')

In [7]:
group_data

,lesson_id,event_id,subject,scheduled_time,user_id,tech_quality,role
0,2e3b1173-7edb-4737-9500-a23d7b852518,38114,phys,2020-01-12,e28351f5-4ccb-4549-8647-d43f2b15e7b8,5.0,pupil
1,2e3b1173-7edb-4737-9500-a23d7b852518,38114,phys,2020-01-12,4df2832a-1d63-4453-9659-43993fc35996,5.0,tutor
2,883e272f-c15b-4bf7-8066-f4a8e1862249,50880,phys,2020-01-12,4df2832a-1d63-4453-9659-43993fc35996,5.0,tutor
3,160d0426-867c-4d68-9bf5-85c75e2ecbf2,50880,phys,2020-01-19,4df2832a-1d63-4453-9659-43993fc35996,5.0,tutor
4,49aa8d13-fab5-461c-a233-3a1cf7f67f90,51450,phys,2020-01-12,4df2832a-1d63-4453-9659-43993fc35996,5.0,tutor
...,...,...,...,...,...,...,...
552,e90f529b-1061-4efd-9bba-f73ecee2ef27,55490,phys,2020-01-19,b323a494-40c0-4522-8d63-4d4d591a1def,4.0,pupil
553,056c3d91-dce7-4071-91d3-1de816ee63a3,51001,hist,2020-01-17,720868c1-3d34-44a7-a23e-6c4a0fac609a,NaN,pupil
554,056c3d91-dce7-4071-91d3-1de816ee63a3,51001,hist,2020-01-17,720868c1-3d34-44a7-a23e-6c4a0fac609a,4.0,pupil
555,aa7702bc-5271-469c-a1c2-b7e814fd7e20,49880,bio,2020-01-16,582e6abb-dd9a-42fa-b441-455eac28327e,5.0,pupil


In [8]:
#сортируем по урокам физики и роли, делаем группировку по дате и user_id, считает среднее (по каждому уроку), сортируем
#и удаляем все, кроме первого значения sheduled_time
answer3 = group_data[(group_data.subject == 'phys') & (group_data.role == 'tutor')]\
.groupby(['scheduled_time','user_id'])['tech_quality'].agg({'mean'}).reset_index()\
.sort_values(['scheduled_time','mean']).drop_duplicates('scheduled_time')

In [9]:
answer1

871853874

In [10]:
answer2

'154.157.157.156'

In [11]:
answer3

,scheduled_time,user_id,mean
4,2020-01-11,8fe03f08-8581-430c-a590-9888ab36deb3,4.200000
10,2020-01-12,696c838e-c054-4e9f-a51a-50bf5660f364,4.833333
13,2020-01-13,2fa2ab62-f1b0-4036-872f-bcfd9a8686ff,5.000000
25,2020-01-14,c6718d0e-976c-4d6c-b0e0-32c770776567,4.000000
26,2020-01-15,603b8641-c6f6-4d89-ac89-88e50d45aa0d,5.000000
30,2020-01-16,2fa2ab62-f1b0-4036-872f-bcfd9a8686ff,4.000000
39,2020-01-17,696c838e-c054-4e9f-a51a-50bf5660f364,4.500000
46,2020-01-18,43efce48-94b2-4412-857f-223d45969008,4.000000
58,2020-01-19,be676776-8366-4c71-8a35-d58014806eb5,4.500000
60,2020-01-20,30a19496-bdaf-461c-abbc-2709ae520201,4.333333
